# Assignment: Segmenting and Clustering Neighborhoods in Toronto 

#### Kaylene Robinson


For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

1.) Start by creating a new Notebook for this assignment.

2.) Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

### Import Libraries

In [15]:
# Library data analysis
import pandas as pd
# Library to handle vectorized data
import numpy as np

# Library for web scraping
import requests
from bs4 import BeautifulSoup

print('Libraries Imported!')

Libraries Imported!


## Part 1
###### Objective: Scrape the Wikipedia page data, clean data, and use .shape method to create dataframe.

In [20]:
#import data from Wikipedia
wiki_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki_text= requests.get(wiki_link).text

#Use BeautifulSoup to parse data
Canada_data = BeautifulSoup(wiki_text, 'lxml')

In [23]:
# extract raw table data inside Wikipedia webpage
table = Canada_data.find('table')

def table_cell(i):
    cells = i.find_all('td')
    row = []
    
    for cell in cells:
        if cell.a:            
            if (cell.a.text):
                row.append(cell.a.text)
                continue
        row.append(cell.string.strip())
        
    return row

def table_row():    
    data = []  
    
    for tr in table.find_all('tr'):
        row = table_cell(tr)
        if len(row) != 3:
            continue
        data.append(row)        
    
    return data

In [31]:
data = table_row()
columns = ['Postcode', 'Borough', 'Neighbourhood']
df_toronto = pd.DataFrame(data, columns=columns)
df_toronto.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [34]:
#Ignore "Not Assigned" boroughs
df2 = df_toronto[df.Borough != 'Not assigned']
df2 = df1.sort_values(by=['Postcode','Borough'])

df2.reset_index(inplace=True)
df2.drop('index',axis=1,inplace=True)
df2.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,Rouge
1,M1B,Scarborough,Malvern
2,M1C,Scarborough,Highland Creek
3,M1C,Scarborough,Rouge Hill
4,M1C,Scarborough,Port Union


In [37]:
#Combine neighbourhoods that share the same postcode
df_postcodes = df2['Postcode']
df_postcodes.drop_duplicates(inplace=True)
df3 = pd.DataFrame(df_postcodes)
df3['Borough'] = '';
df3['Neighbourhood'] = '';


df3.reset_index(inplace=True)
df3.drop('index', axis=1, inplace=True)
df2.reset_index(inplace=True)
df2.drop('index', axis=1, inplace=True)

for i in df3.index:
    for j in df1.index:
        if df3.iloc[i, 0] == df2.iloc[j, 0]:
            df3.iloc[i, 1] = df2.iloc[j, 1]
            df3.iloc[i, 2] = df3.iloc[i, 2] + ',' + df2.iloc[j, 2]
            
for i in df3.index:
    s = df3.iloc[i, 2]
    if s[0] == ',':
        s =s [1:]
    df3.iloc[i,2 ] = s
    
df3.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [36]:
#Checking dataframe shape
df3.shape

(103, 3)